In [5]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn import datasets, linear_model

c:\users\daman\python_all\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\daman\python_all\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\daman\python_all\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\daman\python_all\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
c:\users\daman\python_all\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expect

In [6]:
os.chdir('../')
os.getcwd()

'C:\\Users\\daman\\Desktop\\blops4'

In [7]:
os.listdir()

['.git',
 '.vscode',
 'cwl-data',
 'data_imports',
 'data_outputs',
 'scripts',
 'write_ups']

In [8]:
os.listdir('data_imports')

['proleague1.csv', 'qualifier.csv']

In [9]:
pl1 = pd.read_csv('data_imports/proleague1.csv')

In [10]:
pl1.columns

Index(['Unnamed: 0', 'match id', 'series id', 'end time', 'duration (s)',
       'mode', 'map', 'team', 'player', 'win?', 'score', 'kills', 'deaths',
       '+/-', 'k/d', 'kills per 10min', 'deaths per 10min', 'player score',
       'player spm', 'ekia', 'assists', 'headshots', 'suicides', 'team kills',
       'team deaths', 'kills (stayed alive)', 'hits', 'shots', 'accuracy (%)',
       'num lives', 'time alive (s)', 'avg time per life (s)', 'fave weapon',
       'fave specialist', 'fave scorestreaks', 'hill time (s)',
       'hill captures', 'hill defends', 'snd rounds', 'snd firstbloods',
       'snd firstdeaths', 'snd survives', 'bomb pickups', 'bomb plants',
       'bomb defuses', 'bomb sneak defuses', 'snd 1-kill round',
       'snd 2-kill round', 'snd 3-kill round', 'snd 4-kill round',
       'ctrl rounds', 'ctrl firstbloods', 'ctrl firstdeaths', 'ctrl captures',
       '2-piece', '3-piece', '4-piece', '4-streak', '5-streak', '6-streak',
       '7-streak', '8+-streak'],
      dt

In [11]:
cor_vars = list(pl1.columns[9:32])
cor_vars.remove('num lives')
print(cor_vars, sep=',')
cor_test = pl1[cor_vars][:]

['win?', 'score', 'kills', 'deaths', '+/-', 'k/d', 'kills per 10min', 'deaths per 10min', 'player score', 'player spm', 'ekia', 'assists', 'headshots', 'suicides', 'team kills', 'team deaths', 'kills (stayed alive)', 'hits', 'shots', 'accuracy (%)', 'time alive (s)', 'avg time per life (s)']


In [12]:
cor_test['win?'].replace(('W', 'L'), (1, 0), inplace=True)

Accuracy (%) and avg time per life (s) were imported as objects probably due to the addition of some sort of metric at the end of the value (ie. % or s). In looking at accuracy (%) the issue is the percent sign. When correcting this, something else strange popped up as it looks like the two object data types seem to of imported either all 0's or ?'s. This is something to look into but for now I'm excluding these features.

In [13]:
cor_test['accuracy (%)'].replace('%', '', regex=True)
sum(cor_test['accuracy (%)'] != '0.0')
del cor_test['accuracy (%)']
del cor_test['avg time per life (s)']
cor_test.dtypes

win?                      int64
score                     int64
kills                     int64
deaths                    int64
+/-                       int64
k/d                     float64
kills per 10min         float64
deaths per 10min        float64
player score              int64
player spm              float64
ekia                      int64
assists                   int64
headshots                 int64
suicides                  int64
team kills                int64
team deaths               int64
kills (stayed alive)      int64
hits                      int64
shots                     int64
time alive (s)          float64
dtype: object

In [14]:
cor_mat = cor_test.corr()
print(cor_mat['win?'].nlargest(5), cor_mat['win?'].nsmallest(5))

win?               1.000000
+/-                0.433066
k/d                0.388913
player spm         0.161558
kills per 10min    0.157497
Name: win?, dtype: float64 deaths per 10min   -0.169684
deaths             -0.160669
suicides           -0.018522
headshots           0.026450
ekia                0.061733
Name: win?, dtype: float64


The snd related columns, identifier columns, and correlated to winning columns will be combined to act as the subset master list for further analysis.

In [15]:
snd_cols = [name for name in pl1.columns if 'snd' in name]
key_cols = ['match id', 'mode', 'map', 'team', 'player']
cor_cols = ['win?', '+/-', 'player spm', 'deaths per 10min']
subset = key_cols + cor_cols + snd_cols

In [16]:
cor_df = pl1[subset][:]
cor_df['win?'].replace(('W', 'L'), (1,0), inplace=True)
cor_df[cor_df['mode'] == "Search & Destroy"]
cor_df = cor_df.groupby('team').mean()
correl = cor_df.corr()

In [17]:
print(correl['win?'].nlargest(9))

win?                1.000000
+/-                 0.765922
snd firstbloods     0.574290
snd survives        0.526776
snd 2-kill round    0.510721
snd 1-kill round    0.476422
player spm          0.417772
snd rounds          0.387650
snd 3-kill round    0.370165
Name: win?, dtype: float64


In [64]:
import sklearn
from sklearn.model_selection import train_test_split

cols = ['+/-', 'snd firstbloods', 'snd survives', 'snd 2-kill round', 'snd 1-kill round', 'player spm', 'snd 3-kill round']

X, y = cor_df[cols], cor_df['win?']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33)

In [65]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(normalize=True)
pct_reg = lr.fit(X, y)
y_pred = lr.predict(X_test)

In [66]:
from sklearn.metrics import mean_absolute_error 
y_true = y_test[:]
y_true
mean_absolute_error(y_true, y_pred)

0.043639110679291604

In [67]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(y_test, y_pred)

0.0030746233410908715

In [68]:
from sklearn.metrics import r2_score
r2_score(y_true, y_pred) 

0.7627703240271373

Model seems to be relatively accurate. Need to assess with more data points and also look into more or different features. Used only the nlargest(9) in the model. Need to look at any negatively correlated variables as well as any variables that incorporate covariance between them within the model.